### Nettoyage des descriptions d'activités pour prédiction ape avec du nlp

In [5]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 88.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 795.4/795.4 kB 68.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 103.1 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 835.9/835.9 kB 4.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 106.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 32.1 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 91.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 78.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41/41 [langchain_community]ommunity]lassic]


In [6]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 80.8 MB/s  0:00:00m0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [7]:
!pip install langchain_community

In [30]:
from data.load_idf_10_24 import load_base

In [9]:
data = load_base()

Data Frame idf_10_24 récupéré depuis S3


In [10]:
data = data.loc[data["objet"]!="non-disp"]

In [11]:
from typing import List
import spacy

def clean_docs(
    texts: List[str],
    remove_stopwords: bool = False,
    n_process: int = 4,
    remove_punctuation: bool = True
) -> List[str]:
    """
    Cleans a list of text documents by tokenizing, optionally removing stopwords, and optionally removing punctuation.

    Parameters:
        texts (List[str]): List of text documents to clean.
        remove_stopwords (bool): Whether to remove stopwords. Default is False.
        n_process (int): Number of processes to use for processing. Default is 4.
        remove_punctuation (bool): Whether to remove punctuation. Default is True.

    Returns:
        List[str]: List of cleaned text documents.
    """
    # Load spacy's nlp model
    docs = nlp.pipe(
        texts,
        n_process=n_process,
        disable=['parser', 'ner', 'lemmatizer', 'textcat']
    )

    # Pre-load stopwords for faster checking
    stopwords = set(nlp.Defaults.stop_words)

    # Process documents
    docs_cleaned = (
        ' '.join(
            tok.text.lower().strip()
            for tok in doc
            if (not remove_punctuation or not tok.is_punct) and
               (not remove_stopwords or tok.text.lower() not in stopwords)
        )
        for doc in docs
    )

    return list(docs_cleaned)

In [12]:
nlp = spacy.load('fr_core_news_sm')

In [13]:
data["text_clean"] = clean_docs(data["objet"],remove_stopwords=True,remove_punctuation=True)

In [16]:
data_t = data[["text_clean","codeAPE","siren"]]

In [ ]:
#Connexion à S3
os.environ["AWS_ACCESS_KEY_ID"] = getpass.getpass()
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass.getpass()
os.environ["AWS_SESSION_TOKEN"] = getpass.getpass()
os.environ["AWS_DEFAULT_REGION"] = 'us-east-1'
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])


In [25]:
#Stockage sur S3
MY_BUCKET = "guillaume176"

FILE_PATH_OUT_S3 = f"{MY_BUCKET}/diffusion/data_nlp/data_text.parquet"

with fs.open(FILE_PATH_OUT_S3,"wb") as file_out:
    data_t.to_parquet(file_out, index=False)

print(f"Donnés chargés dans {FILE_PATH_OUT_S3}")


Donnés chargés dans guillaume176/diffusion/data_nlp/data_text.parquet


In [26]:
path_data = "guillaume176/diffusion/data_nlp"
file_list = fs.ls(path_data)[1:]
file_list = ["https://minio.lab.sspcloud.fr/" + f for f in file_list]
print(file_list[0])

https://minio.lab.sspcloud.fr/guillaume176/diffusion/data_nlp/data_text.parquet
